# Importing and preparing supermarkets data

## Libraries and settings

In [118]:
# Libraries
import os
import fnmatch
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print('Current working directory:', os.getcwd())

# Show .json files in the current working directory
flist = fnmatch.filter(os.listdir('.'), '*.json')
for i in flist:
    print(i)

Current working directory: /workspaces/data_analytics/Week_02
supermarkets.json


## Importing data

In [119]:
# Read the data to a pandas data frame
df1 = pd.read_json('supermarkets.json', encoding='utf-8')
df1.head(5)

,type,id,lat,lon,tags
0,node,33126515,47.155616,9.037915,"{'brand': 'Spar', 'brand:wikidata': 'Q610492',..."
1,node,36726161,47.226191,8.980329,"{'addr:city': 'Uznach', 'addr:housenumber': '2..."
2,node,39768209,47.225069,8.969981,"{'addr:city': 'Uznach', 'addr:postcode': '8730..."
3,node,39947904,47.376732,8.542161,"{'addr:city': 'Zürich', 'addr:country': 'CH', ..."
4,node,48932835,47.375020,8.522895,"{'addr:city': 'Zürich', 'addr:housenumber': '7..."


## Count number of rows and columns in the data frame

In [120]:
# Dimension (rows, columns)
print('Dimension:', df1.shape)

# Number of rows
print('Number of rows:', df1.shape[0])

# Number of columns
print('Number of columns:', df1.shape[1])

Dimension: (3392, 5)
Number of rows: 3392
Number of columns: 5


## Column 'tags' is a pandas Series with dictionaries -> change to data frame

In [121]:
# Type of the first item of column tags
print(type(df1.tags))
print(type(df1.tags[0]))

# Content of the first item of column tags
print(df1.tags[0].keys())

# Change to data frame
df2 = pd.DataFrame.from_records(df1.tags)
df2 = df2[['brand', 'opening_hours', 'shop', 'addr:city', 'addr:street', 'addr:housenumber', 'addr:postcode']]

# Rename selected columns
df2 = df2.rename(columns={'addr:city': 'city',
                          'addr:street':'street',
                          'addr:housenumber': 'housenumber',
                          'addr:postcode': 'postcode'})

# Show first records of data frame
df2.head()

<class 'pandas.core.series.Series'>
<class 'dict'>
dict_keys(['brand', 'brand:wikidata', 'brand:wikipedia', 'name', 'opening_hours', 'shop'])


,brand,opening_hours,shop,city,street,housenumber,postcode
0,Spar,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00,supermarket,NaN,NaN,NaN,NaN
1,Migros,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17...",supermarket,Uznach,Zürcherstrasse,25,8730
2,Coop,NaN,supermarket,Uznach,NaN,NaN,8730
3,Coop,Mo-Sa 06:00-22:00,supermarket,Zürich,Bahnhofbrücke,1,8001
4,Migros,Mo-Sa 08:00-21:00; PH off,supermarket,Zürich,Wengistrasse,7,8004


## Merge df1 and df2

In [122]:
# Merge df and df2
df = pd.merge(df1[['type', 'id', 'lat', 'lon']], 
              df2[['brand', 'opening_hours','shop', 'city', 'street', 'housenumber', 'postcode']],
              left_index=True, 
              right_index=True)
df.head(5)

,type,id,lat,lon,brand,opening_hours,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00,supermarket,NaN,NaN,NaN,NaN
1,node,36726161,47.226191,8.980329,Migros,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17...",supermarket,Uznach,Zürcherstrasse,25,8730
2,node,39768209,47.225069,8.969981,Coop,NaN,supermarket,Uznach,NaN,NaN,8730
3,node,39947904,47.376732,8.542161,Coop,Mo-Sa 06:00-22:00,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,Mo-Sa 08:00-21:00; PH off,supermarket,Zürich,Wengistrasse,7,8004


## Count and identify the number of missing values (if any)

In [123]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values, e.g.:
df.loc[pd.isna(df['city'])]

type                0
id                  0
lat                 0
lon                 0
brand            1065
opening_hours    1361
shop                0
city             1777
street           1608
housenumber      1680
postcode         1709
dtype: int64


,type,id,lat,lon,brand,opening_hours,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00,supermarket,NaN,NaN,NaN,NaN
5,node,60271452,47.406671,9.305450,NaN,NaN,supermarket,NaN,NaN,NaN,NaN
6,node,70656485,47.491253,8.733981,NaN,NaN,supermarket,NaN,NaN,NaN,NaN
10,node,81321513,47.532917,9.066408,Landi,"Mo-Sa 08:00-12:00, 13:30-18:00",supermarket,NaN,NaN,NaN,NaN
13,node,95582038,47.050385,9.059214,NaN,Mo-Su 09:00-21:00;PH off,supermarket,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3384,node,11083317088,46.862184,9.531169,Lidl,NaN,supermarket,NaN,NaN,NaN,NaN
3386,node,11098091830,46.205111,6.130174,Coop,NaN,supermarket,NaN,NaN,NaN,NaN
3387,node,11099817248,46.928691,7.561873,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 07:30-17:00,supermarket,NaN,NaN,NaN,NaN
3388,node,11103235832,46.166742,8.771970,Migros,Mo-Fr 07:30-19:00; Th 07:30-20:00; Sa 07:30-18:30,supermarket,NaN,NaN,NaN,NaN


## Count and identify duplicated values (if any)

In [124]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['id']].duplicated()]

0


,type,id,lat,lon,brand,opening_hours,shop,city,street,housenumber,postcode


## Get data types of all variables

In [125]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

type              object
id                 int64
lat              float64
lon              float64
brand             object
opening_hours     object
shop              object
city              object
street            object
housenumber       object
postcode          object
dtype: object

## Übung (Task 3)

### 1. Additional filters on Supermarkets (Brand = Coop / City = Zürich)

In [126]:
df_filtered1 = df.loc[(df['brand'] == 'Coop') & (df['city'] == 'Zürich')]
df_filtered1.head(5)

,type,id,lat,lon,brand,opening_hours,shop,city,street,housenumber,postcode
3,node,39947904,47.376732,8.542161,Coop,Mo-Sa 06:00-22:00,supermarket,Zürich,Bahnhofbrücke,1,8001
9,node,79977755,47.340070,8.530546,Coop,Mo-Sa 07:30-21:00,supermarket,Zürich,Alte Kalchbühlstrasse,15,8038
59,node,265776668,47.376417,8.559594,Coop,Mo-Fr 07:30-21:00; Sa 07:30-20:00,supermarket,Zürich,Zürichbergstrasse,75,8044
63,node,266630559,47.377716,8.511219,Coop,Mo-Sa 07:00-22:00,supermarket,Zürich,Badenerstrasse,333,8003
70,node,267345511,47.385809,8.516574,Coop,Mo-Fr 06:00-20:00; Sa 08:00-20:00,supermarket,Zürich,Maagplatz,1,8005


### 2. Additional filters on Supermarkets (Brand = Mirgos / City = Zürich)

In [127]:
df_filtered2 = df.loc[(df['brand'] == 'Migros') & (df['city'] == 'Zürich')]
df_filtered2.head(5)

,type,id,lat,lon,brand,opening_hours,shop,city,street,housenumber,postcode
4,node,48932835,47.375020,8.522895,Migros,Mo-Sa 08:00-21:00; PH off,supermarket,Zürich,Wengistrasse,7,8004
11,node,83330862,47.344749,8.529981,Migros,Mo-Sa 07:30-21:00; PH off,supermarket,Zürich,Etzelstrasse,3,8038
16,node,119249170,47.375255,8.536107,Migros,Mo-Sa 09:00-20:00; PH off,supermarket,Zürich,Löwenstrasse,31-35,8001
50,node,262400822,47.364072,8.530945,Migros,"Mo-Fr 06:30-22:00; PH,Sa-Su 08:00-22:00",supermarket,Zürich,Tessinerplatz,10,8002
71,node,267346993,47.385598,8.531471,Migros,Mo-Sa 07:30-20:00; PH off,supermarket,Zürich,Limmatstrasse,152,8005


### 3. Additional filters on Supermarkets (Brand = Coop / City = Zürich, Basel, Bern)

In [128]:
df_filtered3 = df.loc[(df['brand'] == 'Coop') & ((df['city'] == 'Zürich') | (df['city'] == 'Basel') | (df['city'] == 'Bern'))]
df_filtered3.head(5)

,type,id,lat,lon,brand,opening_hours,shop,city,street,housenumber,postcode
3,node,39947904,47.376732,8.542161,Coop,Mo-Sa 06:00-22:00,supermarket,Zürich,Bahnhofbrücke,1,8001
9,node,79977755,47.340070,8.530546,Coop,Mo-Sa 07:30-21:00,supermarket,Zürich,Alte Kalchbühlstrasse,15,8038
59,node,265776668,47.376417,8.559594,Coop,Mo-Fr 07:30-21:00; Sa 07:30-20:00,supermarket,Zürich,Zürichbergstrasse,75,8044
63,node,266630559,47.377716,8.511219,Coop,Mo-Sa 07:00-22:00,supermarket,Zürich,Badenerstrasse,333,8003
70,node,267345511,47.385809,8.516574,Coop,Mo-Fr 06:00-20:00; Sa 08:00-20:00,supermarket,Zürich,Maagplatz,1,8005


### 4. Additional filters on Supermarkets (Opening Hours)

In [133]:
df_filtered4 = df.loc[(df['opening_hours'].notna())]
df_filtered4.head(5)

,type,id,lat,lon,brand,opening_hours,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00,supermarket,NaN,NaN,NaN,NaN
1,node,36726161,47.226191,8.980329,Migros,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17...",supermarket,Uznach,Zürcherstrasse,25,8730
3,node,39947904,47.376732,8.542161,Coop,Mo-Sa 06:00-22:00,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,Mo-Sa 08:00-21:00; PH off,supermarket,Zürich,Wengistrasse,7,8004
7,node,70656488,47.491874,8.706448,Migros,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00,supermarket,Winterthur,Zürcherstrasse,102,8406


### Save data to file

In [130]:
df.to_csv('supermarkets_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [131]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1014-azure
Datetime: 2024-09-25 14:12:45
Python Version: 3.11.10
-----------------------------------
